In [1]:
service = dict([('title', 'S1 data selection'),
                ('abstract', 'Sentinel-1 data selection'),
                ('id', 's1_crop')])

In [2]:
base_endpoint = dict([('id', 'base_endpoint'),
              ('value', 'https://catalog.terradue.com/sentinel1/series/SLC_GRDH/search'),
              ('title', 'catalogue base URL'),
              ('abstract', 'base URL of the catalogue')])

In [3]:
start = dict([('id', 'start'),
              ('value', '2018-06-08T00:00:00Z'),
              ('title', 'start date for dataset selection'),
              ('abstract', 'start date for dataset selection')])

In [4]:
stop = dict([('id', 'stop'),
            ('value', '2018-06-08T23:59:59Z'),
            ('title', 'stop date for dataset selection'),
            ('abstract', 'stop date for dataset selection')])

In [5]:
search_type = dict([('id', 'time_filter_type'),
            ('value', 'date'),
            ('title', 'search time filter type'),
            ('abstract', 'search time filter type for dataset selection (date|update)')])

In [6]:
cat_index = dict([('id', 'cat_index'),
              ('value', 'ecop-cnr-issia'),
              ('title', 'publishing catalog index'),
              ('abstract', 'publishing catalog index')])

In [7]:
config_url = dict([('id', 'config_url'),
                   ('value', 'https://store.terradue.com/ec-ecopotential/config/config.ini'),
                   ('title', 'FTP configuration file URL'),
                   ('abstract', 'FTP configuration file URL')])

In [8]:
username = dict([('id', 'username'),
                 ('value', 'ecop-cnr-issia'),
                 ('title', 'username for DA access'),
                 ('abstract', 'username for DA access')])

In [9]:
api_key  = dict([('id', 'api_key'),
                 ('value', 'AKCp5aUjBi3JvjztBDdqPMMjM9beDhtcMLjGsn5axpviB3rAndimLeqfK3bTqUJsR2MtAkjiY'),
                 ('title', 'apikey for DA access'),
                 ('abstract', 'apikey for DA access')])

In [10]:
data_path = '/workspace/data'

In [11]:
input_reference = "donana"

In [27]:
#input_identifier = 'S1A_IW_SLC__1SDV_20180503T182621_20180503T182648_021744_025843_8690'
#input_identifier = 'S1A_IW_SLC__1SDV_20180608T182623_20180608T182650_022269_0268EB_AB32'
input_identifier = 'S1A_IW_GRDH_1SDV_20180621T062707_20180621T062732_022451_026E72_CDDD'

In [28]:
#input_identifier_wkt = 'POLYGON((-8.535965 37.180996,-5.694044 37.580723,-5.365185 35.960056,-8.145734 35.558769,-8.535965 37.180996))'
#input_identifier_wkt = 'POLYGON((-8.536756 37.18034,-5.697594 37.579758,-5.368666 35.959213,-8.146519 35.558239,-8.536756 37.18034))'
input_identifier_wkt='POLYGON((-4.809574 35.99057,-7.706526 36.404522,-7.403009 37.906551,-4.448186 37.494438,-4.809574 35.99057))'

In [29]:
input_pa_name = 'Donana'

In [30]:
input_ftp_server_address = 'sftp://frontend.recas.ba.infn.it/lustre/ecopotential/incoming/PAs'

In [31]:
import os
import shutil
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import snappy
import dateutil.parser as parser
from datetime import datetime

from os.path import basename
import osr
import warnings
warnings.filterwarnings("ignore")
from contextlib import closing
from zipfile import ZipFile, ZIP_DEFLATED
import zipfile
import sys

#sys.path.append('/application/notebook/libexec/')
sys.path.append('/workspace/dcs-s1-crop/src/main/app-resources/notebook/libexec/')
sys.path.append(os.getcwd())
import ISOMetadata

from shapely.wkt import loads

In [32]:
def unzip(source_filename, dest_dir):
    with zipfile.ZipFile(source_filename) as zf:
        zf.extractall(dest_dir)

In [ ]:
unzip('%s/%s.zip' % (data_path,input_identifier), '%s/%s' % (data_path,input_identifier))

In [ ]:
def get_MODE_from_name():
    return input_identifier.split('_')[1]

In [ ]:
def get_pt_from_name():
    return input_identifier.split('_')[2]

In [ ]:
spatial_res = dict()
pixel_spacing = dict()

spatial_res['SLC'] = dict()
spatial_res['GRDM'] = dict()
spatial_res['GRDH'] = dict()

#spatial_res['SLC'] = {'SM':'1.7x4.3 m to 3.6x4.9 m', 'IW':'2.7x22 m to 3.5x22 m','EW':'7.9x43 m to 15x43 m','WV':'2.0x4.8 m and 3.1x4.8 m'}
spatial_res['SLC'] = {'SM':{'a':['1.7','3.6'],'r':['4.3','4.9']}, 'IW':{'a':['2.7','3.5'],'r':['22','22 m']},'EW':{'a':['7.9','15'],'r':['43','43']},'WV':{'a':['2.0','3.1'],'r':['4.8','4.8']}}

#spatial_res=['GRDM'] = {'SM':'84x84', 'IW':'88x87','EW':'93x87','WV':'52x51'}
spatial_res['GRDM'] = {'SM':{'a':['84'],'r':['84']}, 'IW':{'a':['88'],'r':['87']},'EW':{'a':['93'],'r':['87']},'WV':{'a':['52'],'r':['51']}}

#spatial_res=['GRDH'] = {'SM':'23x23', 'IW':'20x22','EW':'50x50'}
spatial_res['GRDH'] = {'SM':{'a':['23'],'r':['23']}, 'IW':{'a':['20'],'r':['22']},'EW':{'a':['50'],'r':['50']}}

pixel_spacing['SLC'] = dict()
pixel_spacing['GRDM'] = dict()
pixel_spacing['GRDH'] = dict()

pixel_spacing['SLC']= {'SM':{'a':['1.5','3.1'],'r':['3.6','4.1']}, 'IW':{'a':['2.3'],'r':['14.1']},'EW':{'a':['5.9'],'r':['19.9']},'WV':{'a':['1.7','2.7'],'r':['4.1','4.1']}}
#pixel_spacing['SLC']= {'SM':['1.5','3.1']?['3.6','4.1'], 'IW':['2.3']?['14.1'],'EW':['5.9']?['19.9'],'WV':['1.7','2.7']?['4.1','4.1']}
pixel_spacing['GRDM'] = {'SM':{'a':['40'],'r':['40']}, 'IW':{'a':['40'],'r':['40']},'EW':{'a':['40'],'r':['40']},'WV':{'a':['25'],'r':['25']}}
#pixel_spacing['GRDM'] = {'SM':['40'], 'IW':['40'],'EW':['40'],'WV':['25']}
pixel_spacing['GRDH'] = {'SM':{'a':['10'],'r':['10']}, 'IW':{'a':['10'],'r':['10']},'EW':{'a':['25'],'r':['25']}}
#pixel_spacing['GRDH'] = {'SM':['10'], 'IW':['10'],'EW':['25']}


In [33]:
s1prd = "%s/%s/%s.SAFE/manifest.safe" % (data_path, input_identifier, input_identifier)
product = snappy.ProductIO.readProduct(s1prd)

width = product.getSceneRasterWidth()
height = product.getSceneRasterHeight()
prod_type = product.getProductType()
name = product.getName()
description = product.getDescription()
band_names = product.getBandNames()

#prod_type = get_pt_from_name()
#prod_mode = get_MODE_from_name()
#print prod_mode
#print prod_type
#print pixel_spacing[prod_type][prod_mode]


  band name:     Amplitude_VH
  band name:     Intensity_VH
  band name:     Amplitude_VV
  band name:     Intensity_VV


In [34]:
product_date = parser.parse(product.getStartTime().toString()).date()
START_DATE = '%sZ' %parser.parse(product.getStartTime().toString()).isoformat()
END_DATE = '%sZ' %parser.parse(product.getEndTime().toString()).isoformat()

now = datetime.now()

metadata_date = now.strftime("%Y-%m-%d")
metadata_year = now.strftime("%Y")

creation_date = START_DATE

date_path = '%s/%02d/%02d' % (product_date.year, product_date.month, product_date.day)
    
middle_path = '%s/EO_Data/Sentinel1/Raw'%input_pa_name
        
ftp_remote_path = os.path.join(input_ftp_server_address, os.path.join(middle_path,date_path))
output_name = name+'.zip'
download_URL = '%s/%s' %(ftp_remote_path, output_name)


print 'ftp_remote_path: ',ftp_remote_path
print 'download_URL: ',download_URL

####################################################################
#start_Date = parser.parse(product.getStartTime().toString()).date()

#end_Date = parser.parse(product.getEndTime().toString()).date()

#now = datetime.now()

#metadata_date = now.strftime("%Y-%m-%d")
#creation_date = start_Date


ftp_remote_path:  sftp://frontend.recas.ba.infn.it/lustre/ecopotential/incoming/PAs/Donana/EO_Data/Sentinel1/Raw/2018/06/21
download_URL:  sftp://frontend.recas.ba.infn.it/lustre/ecopotential/incoming/PAs/Donana/EO_Data/Sentinel1/Raw/2018/06/21/S1A_IW_GRDH_1SDV_20180621T062707_20180621T062732_022451_026E72_CDDD.zip


In [35]:
def set_dates():
    
    iso_metadata.set_start_date(START_DATE)
    iso_metadata.set_end_date(END_DATE)

In [36]:
def set_info(short_name, title, abstract):
    
    iso_metadata.set_identifier(short_name)
    iso_metadata.set_title(title)
    iso_metadata.set_abstract(abstract)

In [37]:
def set_geo(band_name,wkt):
    
    sp_i = dict()
    #ds = gdal.Open(geotiff)

    band = product.getBand(band_name)
    
    sp_i['dims'] = str(band.getDataElemSize())
    print 'band %s - dims %s' %(band_name,sp_i['dims'])
    col_size = band.getRasterWidth()
    row_size = band.getRasterHeight()
    
    iso_metadata.set_col_size(str(col_size))
    iso_metadata.set_row_size(str(row_size))
    
    sp_i['col_size'] = str(col_size)
    sp_i['row_size'] = str(row_size)
    
    polygon = loads(wkt)
    
    #min_x,min_y,max_x,max_y = polygon.bounds
    
    lr_x,lr_y,ul_x,ul_y = polygon.bounds
    
    nw_corner = '%s %s' % (str(ul_x), str(ul_y))
    se_corner = '%s %s' % (str(lr_x), str(lr_y))
    
    sp_i['nw_corner'] = nw_corner

    
    sp_i['se_corner'] = se_corner
    
    iso_metadata.set_se_corner(se_corner)
    
    # adding single band spatial info to the global spatial info dict for the general ISOmetadata
    spatial_info[band_name] = sp_i

In [38]:
def get_minmax_lonlat(wkt):
       
    
    #POLYGON((-26.4804 33.505,-26.7143 32.3182,-24.1248 32.0496,-23.8554 33.2363,-26.4804 33.505))
    polygon = loads(wkt)
    
    min_x,min_y,max_x,max_y = polygon.bounds
    
    return (min_x,min_y,max_x,max_y)

In [39]:
iso_metadata = ISOMetadata.ISOMetadata()
iso_metadata.set_organisation('CNR')
iso_metadata.set_contact('info@terradue.com')
iso_metadata.set_date(metadata_date)
iso_metadata.set_creation_date(creation_date)
iso_metadata.set_data_format('SAFE')
iso_metadata.set_data_type('CInt16')
iso_metadata.set_responsible_party('CNR')
iso_metadata.set_onlineResource(download_URL)
iso_metadata.set_pa('Area covering %s PA' %input_pa_name)

lineage_template = 'Terradue [%s], Copernicus Sentinel data [%s]. \
This work has received funding from the European Union\'s Horizon 2020 research and innovation programme \
under grant agreement No 641762 (ECOPOTENTIAL: improving future ecosystem benefits through earth observations)'
lineage_str = lineage_template %(str(metadata_year), str(product_date.year))
#print lineage_str
iso_metadata.set_lineage_template(lineage_str)

setting:  CNR
setting:  CNR
setting:  info@terradue.com
setting:  info@terradue.com
setting:  2018-09-18
setting:  2018-06-21T06:27:07.575901Z
setting:  SAFE
setting:  CInt16
setting:  CNR
setting:  sftp://frontend.recas.ba.infn.it/lustre/ecopotential/incoming/PAs/Donana/EO_Data/Sentinel1/Raw/2018/06/21/S1A_IW_GRDH_1SDV_20180621T062707_20180621T062732_022451_026E72_CDDD.zip
setting:  Area covering Donana PA
Terradue [2018], Copernicus Sentinel data [2018]. This work has received funding from the European Union's Horizon 2020 research and innovation programme under grant agreement No 641762 (ECOPOTENTIAL: improving future ecosystem benefits through earth observations)
setting:  Terradue [2018], Copernicus Sentinel data [2018]. This work has received funding from the European Union's Horizon 2020 research and innovation programme under grant agreement No 641762 (ECOPOTENTIAL: improving future ecosystem benefits through earth observations)


### Writing General XML

In [40]:
set_dates()
    
short_name = name
title = 'Sentinel-1 SAR Standard L1 %s Product %s ' % (prod_type, name)

abstract = 'Sentinel-1 SAR Standard L1 %s Product covering %s ' % (prod_type, input_pa_name)
        
set_info(short_name, title, abstract)

spatial_info = dict()

for bname in list(band_names):
    set_geo(bname, input_identifier_wkt)


#iso_metadata.set_spatial_resolutions(spatial_res[prod_type][prod_mode]['a'?'r'])

iso_metadata.set_spatialReprInfo_elems(spatial_info, list(band_names))

min_x,min_y,max_x,max_y = get_minmax_lonlat(input_identifier_wkt)

iso_metadata.set_min_lon(str(min_x))
iso_metadata.set_min_lat(str(min_y))
iso_metadata.set_max_lon(str(max_x))
iso_metadata.set_max_lat(str(max_y))

iso_metadata.write(name + '.xml')


setting:  2018-06-21T06:27:07.575901Z
setting:  2018-06-21T06:27:32.573875Z
setting:  S1A_IW_GRDH_1SDV_20180621T062707_20180621T062732_022451_026E72_CDDD
setting:  S1A_IW_GRDH_1SDV_20180621T062707_20180621T062732_022451_026E72_CDDD
setting:  Sentinel-1 SAR Standard L1 GRD Product S1A_IW_GRDH_1SDV_20180621T062707_20180621T062732_022451_026E72_CDDD 
setting:  Sentinel-1 SAR Standard L1 GRD Product covering Donana 
band Amplitude_VH - dims 4
setting:  26457
setting:  16685
setting:  -8.536756 35.558239
band Intensity_VH - dims 4
setting:  26457
setting:  16685
setting:  -8.536756 35.558239
band Amplitude_VV - dims 4
setting:  26457
setting:  16685
setting:  -8.536756 35.558239
band Intensity_VV - dims 4
setting:  26457
setting:  16685
setting:  -8.536756 35.558239
setting:  4
setting:  -5.368666 37.579758
setting:  -8.536756 35.558239
setting:  16685
setting:  26457
setting:  -8.536756
setting:  35.558239
setting:  -5.368666
setting:  37.579758


### Moving the original zip archive into the runtime dir

In [41]:
shutil.move('%s/%s.zip' % (data_path,input_identifier),'./%s.zip' %input_identifier)

IOError: [Errno 2] No such file or directory: '/workspace/data/S1A_IW_GRDH_1SDV_20180621T062707_20180621T062732_022451_026E72_CDDD.zip'